In [1]:
#With Transshipment Nodes
using JuMP
import DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中	仙台青	蒲町小	長町中	七郷小	南光台	新田小	仙台高	錦ケ丘	上杉山	宮城野	仙台商	岩切小	榴岡小	東長町	富沢小	富沢中	TSSW	TSDA	TSMP	TSTP	TSMN	TSHW	TSMS	TSH1	TSH2	TSH3  EVACUEES
          TTWR .	.	.	2.85	.	.	.	4.80	.	1.65	2.31	.	.	1.47	4.33	.	4.38	1.39	4.37	3.94	1.99	.	3.88	2.46	3.76	4.34	4.49	894
          ZUHO .	.	.	2.30	.	.	.	4.58	.	2.36	3.37	.	.	2.54	4.12	4.54	3.57	1.20	.	3.00	3.03	.	4.03	3.44	3.85	4.53	4.64	60
          OSAK .	2.05	.	.	.	.	.	1.46	.	2.65	.	.	.	4.45	.	.	.	2.06	4.17	.	.	.	.	.	.	.	.	100000
          TDAI .	.	.	2.54	.	.	.	4.82	.	1.93	2.61	.	.	1.78	4.11	4.98	4.03	1.35	4.69	3.57	2.27	.	3.77	2.70	3.63	4.25	4.39	5000
          AERT .	.	.	3.56	.	4.54	4.64	4.84	.	1.20	1.77	.	.	0.93	4.86	.	.	1.75	3.70	4.72	1.50	.	4.22	2.06	4.13	4.64	4.81	584
          TEPC .	4.81	.	3.86	.	4.33	4.78	4.47	.	0.78	2.04	.	.	1.25	.	.	.	1.58	3.36	4.92	1.82	.	4.64	2.40	4.54	.	.	12500
          SIPO .	.	.	.	.	.	.	.	.	.	.	3.68	.	.	.	.	.	.	.	.	.	4.29	.	.	.	.	.	1653
          MEDI .	3.88	.	3.89	.	4.99	.	3.45	.	0.99	3.21	.	.	2.39	.	.	.	0.57	3.65	4.54	2.97	.	.	3.53	.	.	.	346
          SSTA .	.	4.83	3.35	.	4.71	4.60	.	.	1.47	1.66	.	.	0.81	4.61	.	4.96	1.87	3.94	4.58	1.36	.	3.95	1.90	3.86	4.37	4.54	75608
          AQUA .	.	.	.	.	.	4.97	.	.	.	.	.	4.19	.	.	.	.	.	.	.	.	.	.	.	.	.	.	913
          MOPS .	.	.	.	.	.	.	.	.	.	.	.	5.09	.	.	.	.	.	.	.	.	.	.	.	.	.	.	10000
          YURT .	.	.	.	.	2.70	.	.	.	.	.	1.03	.	.	.	.	.	.	2.58	.	.	0.49	.	.	.	.	.	11467
          CAST .	4.51	.	2.90	.	.	.	3.87	.	2.68	4.27	.	.	3.42	4.85	4.72	3.77	1.15	.	3.01	3.95	.	4.95	4.41	4.76	.	.	87
          RAKU .	.	2.96	3.98	4.13	4.78	3.19	.	.	3.13	0.68	.	.	1.21	4.39	.	.	3.79	4.74	.	0.67	.	3.24	0.13	3.27	3.48	3.68	27000
          TLSW 9.36	4.01	6.62	3.51	7.86	5.56	6.33	3.49	9.50	1.55	3.51	7.02	8.45	2.66	5.30	5.65	4.69	0.00	4.21	4.02	3.24	6.61	5.10	3.77	4.95	5.59	5.72   999999
          TLDA .	4.38	7.45	7.22	8.38	1.86	4.90	4.57	.	2.80	4.08	3.21	5.62	3.91	8.53	9.66	8.71	4.23	0.00	8.16	4.17	2.66	7.74	4.63	7.69	8.09	8.28   999999
          TLMP .	7.36	7.04	1.70	8.21	9.21	8.68	6.67	9.68	5.35	5.76	.	.	5.11	3.11	1.75	0.89	4.02	8.13	0.00	5.41	.	4.03	5.60	3.80	4.45	4.45   999999
          TLTP .	6.56	3.63	3.96	4.79	4.38	3.37	6.26	.	2.47	0.36	7.35	5.98	0.58	4.66	6.50	5.66	3.23	4.16	5.47	0.00	6.78	3.64	0.58	3.64	3.95	4.14   999999
          TLMN .	5.20	9.87	9.82	.	3.07	6.79	5.70	.	5.35	6.67	0.62	6.40	6.57	.	.	.	6.62	2.66	.	6.80	0.00	.	7.22	.	.	.   999999
          TLHW .	9.02	3.23	2.47	4.29	8.01	5.98	8.50	.	5.38	3.86	.	8.86	3.81	1.38	4.04	3.62	5.05	7.72	4.03	3.64	.	0.00	3.37	0.23	0.50	0.62   999999
          TLMS .	7.13	3.05	4.07	4.20	4.65	3.12	6.83	.	3.05	0.55	7.77	5.86	1.14	4.51	6.56	5.77	3.76	4.61	5.66	0.58	7.19	3.37	0.00	3.40	3.62	3.81   999999
          TLH1 .	8.88	3.43	2.24	4.50	8.02	6.09	8.35	.	5.29	3.87	.	8.96	3.78	1.23	3.84	3.40	4.90	7.68	3.80	3.64	.	0.23	3.40	0.00	0.71	0.79   999999
          TLH2 .	9.49	3.02	2.94	3.99	8.27	6.04	8.98	.	5.80	4.13	.	8.92	4.17	1.59	4.32	3.98	5.53	8.07	4.45	3.95	.	0.50	3.62	0.71	0.00	0.20   999999
          TLH3 .	9.63	3.16	2.97	4.09	8.46	6.23	9.11	.	5.97	4.33	.	9.10	4.35	1.50	4.23	3.94	5.66	8.25	4.44	4.14	.	0.62	3.81	0.79	0.20	0.00   999999
        VACANCIES 15232 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 999999 999999 999999 999999 999999 999999 999999 999999 999999 999999 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

print("\nSolution:\n")
solve_transportation_problem(data)

#write for loop

#########################################

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(data, epsilon)
println("\nNumerical Derivative Matrix:")
for row in eachrow(derivative_matrix)
    println(row)
end

2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA"  …  "TLDA", "TLMP", "TLTP", "TLMN", "TLHW", "TLMS", "TLH1", "TLH2", "TLH3", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "南光台", "新田小", "仙台高", "錦ケ丘", "上杉山"  …  "TSDA", "TSMP", "TSTP", "TSMN", "TSHW", "TSMS", "TSH1", "TSH2", "TSH3", "EVACUEES"]
And data, a 25×28 Matrix{Any}:
      "."       "."       "."      2.85       "."       "."       "."      4.8        "."      1.65      2.31       "."       "."      1.47      4.33       "."      4.38       1.39       4.37       3.94       1.99        "."       3.88       2.46       3.76       4.34       4.49     894
      "."       "."       "."      2.3        "."       "."       "."      4.58       "."      2.36      3.37       "."       "."      2.54      4.12      4.54      3.57       1.2         "."       3.0        3.03        "."       4.03       3.44       3.85

YURT      .      .      .      .      .      .      .      .      .      .      . 11467.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
CAST      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .   87.0      .      .      .      .      .      .      .      .      .      .      .
RAKU      .      . 12950.0      . 12195.0      . 1855.0      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .      .
TLSW 15065.0      .      .      .      .      .      .      .      .      .      .      .      .      .      . 14930.0 18518.0 951486.0      .      .      .      .      .      .      .      .      .
TLDA      .      .      .      .      . 13261.0      .      .      .      .      . 11208.0      .      .      .      .      .      . 975530.0      .      .      .      .      .      .      .      .
TLMP      .      .

In [18]:
#no transshipment nodes
using JuMP
import DelimitedFiles
import HiGHS

open(joinpath(@__DIR__, "transp.txt"), "w") do io
    print(
        io,
        """
             . 広瀬中   仙台青   蒲町小  長町中  七郷小      新田小  仙台高   南光台   錦ケ丘   上杉山   宮城野   仙台商   岩切小    榴岡小  東長町    富沢小  富沢中  EVACUEES
          TTWR 10.71	5.25	5.23	2.85	6.46	5.29	4.80	5.31	10.79	1.65	2.31	7.46	7.68	1.47	4.33	5.32	4.38  1178
          ZUHO 9.99 	5.15	5.96	2.30	7.21	6.35	4.58	6.27	9.92	2.36	3.37	8.10	8.76	2.54	4.12	4.54	3.57  60
          OSAK 7.65 	2.05	8.60	5.35	9.81	7.72	1.46	5.91	8.07	2.65	5.24	6.18	9.36	4.45	7.26	7.13	6.21  100000
          TDAI 10.57	5.32	5.34	2.54	6.59	5.60	4.82	5.66	10.59	1.93	2.61	7.75	8.03	1.78	4.11	4.98	4.03  5000
          AERT 11.05	5.20	5.04	3.56	6.24	4.64	4.84	4.54	11.26	1.20	1.77	6.85	6.94	0.93	4.86	6.07	5.13  773
          TEPC 10.77	4.81	5.43	3.86	6.61	4.78	4.47	4.33	11.03	0.78	2.04	6.49	6.95	1.25	5.24	6.33	5.39  12500
          SIPO 10.21	5.89	13.99	12.75	14.91	11.12	6.61	7.37	11.83	8.49	10.52	3.68	10.65	10.19	14.46	14.79	13.85 1653
          MEDI 9.60 	3.88	6.50	3.89	7.71	5.92	3.45	4.99	9.85	0.99	3.21	6.51	7.95	2.39	5.59	6.14	5.17  346
          SSTA 11.20	5.44	4.83	3.35	6.04	4.60	5.06	4.71	11.37	1.47	1.66	7.10	6.99	0.81	4.61	5.88	4.96  75608
          AQUA 20.33	13.61	6.51	11.64	5.73	4.97	13.62	7.72	20.85	10.09	7.88	10.96	4.19	8.75	11.16	13.84	13.25 913
          MOPS 21.04	14.37	6.77	12.06	5.86	5.67	14.36	8.57	21.52	10.78	8.48	11.84	5.09	9.35	11.45	14.18	13.63 10000
          YURT 12.14	5.55	9.58	9.79	10.35	6.41	6.02	2.70	13.27	5.35	6.47	1.03	5.92	6.43	11.09	12.22	11.26 11467
          CAST 8.98 	4.51	6.99	2.90	8.24	7.21	3.87	6.69	8.90	2.68	4.27	8.07	9.47	3.42	4.85	4.72	3.77  87
          RAKU 13.11	7.21	2.96	3.98	4.13	3.19	6.90	4.78	13.26	3.13	0.68	7.90	5.95	1.21	4.39	6.46	5.68  27000
        VACANCIES 15232 14955 12950 16378 12195 13261 21261 10464 14474 14445 14733 24328 16014 13779 12595 15111 18578 0
        """,
    )
    return
end

function read_data(filename::String)
    data = DelimitedFiles.readdlm(filename)
    rows, columns = data[2:end, 1], data[1, 2:end]
    return Containers.DenseAxisArray(data[2:end, 2:end], rows, columns)
end

data = read_data(joinpath(@__DIR__, "transp.txt"))
print(data)

function solve_transportation_problem(data::Containers.DenseAxisArray)
    # Get the set of supplies and demands
    O, D = axes(data)
    # Drop the EVACUEES and VACANCIES nodes from our sets
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    model = Model(HiGHS.Optimizer)
    set_silent(model)
    @variable(model, x[o in O, d in D] >= 0)
    # Remove arcs with "." cost by fixing them to 0.0.
    for o in O, d in D
        if data[o, d] == "."
            fix(x[o, d], 0.0; force = true)
        end
    end
    @objective(
        model,
        Min,
        sum(data[o, d] * x[o, d] for o in O, d in D if data[o, d] != "."),
    )
    @constraint(model, [o in O], sum(x[o, :]) == data[o, "EVACUEES"])
    @constraint(model, [d in D], sum(x[:, d]) <= data["VACANCIES", d])
    optimize!(model)
    # Pretty print the solution in the format of the input
    print("    ", join(lpad.(D, 7, ' ')))
    for o in O
        print("\n", o)
        for d in D
            if isapprox(value(x[o, d]), 0.0; atol = 1e-6)
                print("      .")
            else
                print(" ", lpad(value(x[o, d]), 6, ' '))
            end
        end
    end
    return
end

print("\nSolution:\n")
solve_transportation_problem(data)

#########################################

function calculate_numerical_derivative(data::JuMP.Containers.DenseAxisArray{Any, 2}, epsilon::Float64 = 1)
    O, D = axes(data)
    O, D = setdiff(O, ["VACANCIES"]), setdiff(D, ["EVACUEES"])
    num_rows, num_cols = length(O), length(D)
    derivative_matrix = zeros(Float64, num_rows, num_cols)

    for (i, o) in enumerate(O), (j, d) in enumerate(D)
        original_cost = data[o, d]

        if original_cost isa Number
            # Increase the cost slightly and re-solve
            data[o, d] = original_cost + epsilon
            increased_cost_model = Model(HiGHS.Optimizer)
            set_silent(increased_cost_model)
            @variable(increased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                increased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(increased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(increased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(increased_cost_model)
            increased_cost_objective = objective_value(increased_cost_model)

            # Decrease the cost slightly and re-solve
            data[o, d] = original_cost - epsilon
            decreased_cost_model = Model(HiGHS.Optimizer)
            set_silent(decreased_cost_model)
            @variable(decreased_cost_model, x[ii in O, jj in D] >= 0)

            for ii in O, jj in D
                if data[ii, jj] == "."
                    fix(x[ii, jj], 0.0; force = true)
                end
            end

            @objective(
                decreased_cost_model,
                Min,
                sum(data[ii, jj] * x[ii, jj] for ii in O, jj in D if data[ii, jj] isa Number),
            )

            @constraint(decreased_cost_model, [ii in O], sum(x[ii, :]) == data[ii, "EVACUEES"])
            @constraint(decreased_cost_model, [jj in D], sum(x[:, jj]) <= data["VACANCIES", jj])

            optimize!(decreased_cost_model)
            decreased_cost_objective = objective_value(decreased_cost_model)

            # Calculate the numerical derivative
            derivative = (increased_cost_objective - decreased_cost_objective) / (2 * epsilon)
            derivative_matrix[i, j] = derivative

            # Reset the original cost
            data[o, d] = original_cost
        end
    end

    return derivative_matrix
end

# Calculate and print the numerical derivative matrix
epsilon = 1.0
derivative_matrix = calculate_numerical_derivative(data, epsilon)
println("\nNumerical Derivative Matrix:")
for row in eachrow(derivative_matrix)
    println(row)
end

2-dimensional DenseAxisArray{Any,2,...} with index sets:
    Dimension 1, Any["TTWR", "ZUHO", "OSAK", "TDAI", "AERT", "TEPC", "SIPO", "MEDI", "SSTA", "AQUA", "MOPS", "YURT", "CAST", "RAKU", "VACANCIES"]
    Dimension 2, Any["広瀬中", "仙台青", "蒲町小", "長町中", "七郷小", "新田小", "仙台高", "南光台", "錦ケ丘", "上杉山", "宮城野", "仙台商", "岩切小", "榴岡小", "東長町", "富沢小", "富沢中", "EVACUEES"]
And data, a 15×18 Matrix{Any}:
    10.71      5.25      5.23      2.85      6.46      5.29      4.8       5.31     10.79      1.65      2.31      7.46      7.68      1.47      4.33      5.32      4.38    1178
     9.99      5.15      5.96      2.3       7.21      6.35      4.58      6.27      9.92      2.36      3.37      8.1       8.76      2.54      4.12      4.54      3.57      60
     7.65      2.05      8.6       5.35      9.81      7.72      1.46      5.91      8.07      2.65      5.24      6.18      9.36      4.45      7.26      7.13      6.21  100000
    10.57      5.32      5.34      2.54      6.59      5.6       4.82      5.66 

In [1]:
#DISTANCE CALCULATOR 1
using Distances

function distance(lat, lng, lat0, lng0)
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0) * cos(lat0)
    return deglen * sqrt(x*x + y*y)
end

#Data
locations_dict = Dict("Zuihōden Temple" => (38.25076685689052, 140.8667686177859),
"Sendai Castle" => (38.25209, 140.85592),
"Sendai Station" => (38.26028322570647, 140.88239995371939),
"Rakuten Seimei Park Miyagi" => (38.25695841274225, 140.90267074199306),
"AER" => (38.26243, 140.88104),
"Osaki Hachiman Shrine" => (38.27251, 140.84503),
"Sendai Umino-Mori Aquarium" => (38.280300194102374, 140.98200323180546),
"Sendai Trust Tower" => (38.25655, 140.87626),
"Yurtec Stadium Sendai" => (38.31933922945424, 140.88186037724955),
"Sendai Izumi PREMIUM OUTLETS" => (38.34201389944614, 140.83717960908587),
"Sendai Mediatheque" => (38.26536, 140.86568),
"Mitsui Outlet Park Sendai Port" => (38.27518, 140.98983),
"Tohoku University Katahira Campus" => (38.25383, 140.87407),
"Tohoku Electric Power Company" => (38.26555, 140.87835),
"Transshipment-location Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-location Sendai West Park" => (38.26098, 140.86235),
"Transshipment-location Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-location Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-location Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-location Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-location Mikamine Park" => (38.2248, 140.8574),
"Transshipment-location Moomin Nanakita Park" => (38.31981, 140.87641))

shelters_dict = Dict("錦ケ丘小" => (38.25644, 140.76093),
"上杉山通小" => (38.27135, 140.87361),
"広瀬中" => (38.27389, 140.76242),
"仙台高" => (38.2821, 140.83406),
"仙台青陵中等教育" => (38.28862, 140.83386),
"榴岡小" => (38.26151, 140.89091),
"岩切小" => (38.29979, 140.94251),
"新田小" => (38.27628, 140.92843),
"宮城野中" => (38.26269, 140.90001),
"七郷小" => (38.24103, 140.94222),
"蒲町小" => (38.24245, 140.92912),
"東長町小" => (38.21892, 140.8892),
"富沢小" => (38.20972, 140.86297),
"長町中" => (38.23077, 140.87384),
"富沢中" => (38.21849, 140.86325),
"南光台小" => (38.30032, 140.9006),
"仙台商業高" => (38.32415, 140.87201),
"Transshipment-shelter Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-shelter Sendai West Park" => (38.26098, 140.86235),
"Transshipment-shelter Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-shelter Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-shelter Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-shelter Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-shelter Mikamine Park" => (38.2248, 140.8574),
"Transshipment-shelter Moomin Nanakita Park" => (38.31981, 140.87641))

#Calculate
distances_dict = Dict{String, Dict{String, Float64}}()

for (loc_name, loc_coords) in locations_dict
    distances_dict[loc_name] = Dict{String, Float64}()
    
    for (sh_name, sh_coords) in shelters_dict
        distances_dict[loc_name][sh_name] = distance(loc_coords[1], loc_coords[2], sh_coords[1], sh_coords[2])
    end
end

#Print
for (loc_name, distances) in distances_dict
    println("Distances from $loc_name:")
    
    for (sh_name, distance) in distances
        println("  $sh_name: $distance km")
    end
    
    println()
end

Distances from Sendai Trust Tower:
  Transshipment-shelter Sendai West Park: 1.3866670908535144 km
  広瀬中: 10.706165212457629 km
  仙台青陵中等教育: 5.253488264906729 km
  Transshipment-shelter Hirosegawanakagawara Ryokuchi Park: 4.342632564801423 km
  Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park: 3.755061197714097 km
  蒲町小: 5.225136877733076 km
  長町中: 2.851534005846954 km
  七郷小: 6.460850045301224 km
  南光台小: 5.308855294814546 km
  新田小: 5.288125807409881 km
  仙台高: 4.797987507462382 km
  Transshipment-shelter Dainoharashinrinkoen: 4.3675234947596095 km
  錦ケ丘小: 10.790975241507619 km
  上杉山通小: 1.6500828140074453 km
  宮城野中: 2.3146910899436053 km
  Transshipment-shelter Mikamine Park: 3.9354660649197175 km
  仙台商業高: 7.462580016934648 km
  岩切小: 7.683293427436732 km
  Transshipment-shelter Moomin Nanakita Park: 6.9744279744032305 km
  榴岡小: 1.4717555808435177 km
  Transshipment-shelter Tsutsujigaoka Park: 1.9898307396072181 km
  東長町小: 4.329538796042483 km
  富沢小: 5.318907136556792 km
  Trans

  Transshipment-shelter Dainoharashinrinkoen: 3.362830527248163 km
  錦ケ丘小: 11.032335498875323 km
  上杉山通小: 0.775828198219253 km
  宮城野中: 2.0431833164628483 km
  Transshipment-shelter Mikamine Park: 4.916883321205494 km
  仙台商業高: 6.485468437239356 km
  岩切小: 6.949993908562245 km
  Transshipment-shelter Moomin Nanakita Park: 5.984694684273462 km
  榴岡小: 1.253279788060108 km
  Transshipment-shelter Tsutsujigaoka Park: 1.8234373138695448 km
  東長町小: 5.244742543621109 km
  富沢小: 6.330537446525945 km
  Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park: 4.636423400145812 km
  Transshipment-shelter Miyaginohara Sports Park: 2.4023744179947317 km
  Transshipment-shelter Hirose River Iida Green Space: 5.231993792905879 km
  富沢中: 5.38589301920371 km

Distances from Transshipment-location Mikamine Park:
  Transshipment-shelter Sendai West Park: 4.015492175053694 km
  広瀬中: 10.321648547165989 km
  仙台青陵中等教育: 7.359426071829252 km
  Transshipment-shelter Hirosegawanakagawara Ryokuchi Park: 4.45340800

  Transshipment-shelter Miyaginohara Sports Park: 8.372529813838067 km
  Transshipment-shelter Hirose River Iida Green Space: 9.975128771654358 km
  富沢中: 13.632290939983735 km

Distances from Yurtec Stadium Sendai:
  Transshipment-shelter Sendai West Park: 6.686649979838516 km
  広瀬中: 12.135147151546489 km
  仙台青陵中等教育: 5.551576361430548 km
  Transshipment-shelter Hirosegawanakagawara Ryokuchi Park: 10.573931971984521 km
  Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park: 10.211915205281235 km
  蒲町小: 9.578721284848323 km
  長町中: 9.794458969827422 km
  七郷小: 10.346126478852183 km
  南光台小: 2.7017940868199894 km
  新田小: 6.406953211520052 km
  仙台高: 6.017621606982947 km
  Transshipment-shelter Dainoharashinrinkoen: 2.5769062257056126 km
  錦ケ丘小: 13.270940272971876 km
  上杉山通小: 5.345788759660722 km
  宮城野中: 6.470590874086619 km
  Transshipment-shelter Mikamine Park: 10.68078108887895 km
  仙台商業高: 1.0280646643456617 km
  岩切小: 5.9222615074808465 km
  Transshipment-shelter Moomin Nanakita Park:

  蒲町小: 3.63484919677622 km
  長町中: 3.9647512195412897 km
  七郷小: 4.787010230620376 km
  南光台小: 4.383513272469631 km
  新田小: 3.3739314006778036 km
  仙台高: 6.257923861261894 km
  Transshipment-shelter Dainoharashinrinkoen: 4.161849726341583 km
  錦ケ丘小: 12.74193613369052 km
  上杉山通小: 2.468683286650202 km
  宮城野中: 0.3567252739669979 km
  Transshipment-shelter Mikamine Park: 5.469468383094419 km
  仙台商業高: 7.347196723247335 km
  岩切小: 5.976584120881784 km
  Transshipment-shelter Moomin Nanakita Park: 6.777127048848962 km
  榴岡小: 0.5773505519745514 km
  Transshipment-shelter Tsutsujigaoka Park: 0.0 km
  東長町小: 4.663362715865789 km
  富沢小: 6.502319001938733 km
  Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park: 3.644014171026461 km
  Transshipment-shelter Miyaginohara Sports Park: 0.5846351826749574 km
  Transshipment-shelter Hirose River Iida Green Space: 4.141369813217257 km
  富沢中: 5.663267097028199 km

Distances from Transshipment-location Hirose River Iida Green Space:
  Transshipment-shelter

In [2]:
#DISTANCE CALCULATOR 2
using Distances

function distance(lat, lng, lat0, lng0)
    deglen = 110.25
    x = lat - lat0
    y = (lng - lng0) * cos(lat0)
    return deglen * sqrt(x*x + y*y)
end

#Data
locations_dict = Dict("Zuihōden Temple" => (38.25076685689052, 140.8667686177859),
"Sendai Castle" => (38.25209, 140.85592),
"Sendai Station" => (38.26028322570647, 140.88239995371939),
"Rakuten Seimei Park Miyagi" => (38.25695841274225, 140.90267074199306),
"AER" => (38.26243, 140.88104),
"Osaki Hachiman Shrine" => (38.27251, 140.84503),
"Sendai Umino-Mori Aquarium" => (38.280300194102374, 140.98200323180546),
"Sendai Trust Tower" => (38.25655, 140.87626),
"Yurtec Stadium Sendai" => (38.31933922945424, 140.88186037724955),
"Sendai Izumi PREMIUM OUTLETS" => (38.34201389944614, 140.83717960908587),
"Sendai Mediatheque" => (38.26536, 140.86568),
"Mitsui Outlet Park Sendai Port" => (38.27518, 140.98983),
"Tohoku University Katahira Campus" => (38.25383, 140.87407),
"Tohoku Electric Power Company" => (38.26555, 140.87835),
"Transshipment-location Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-location Sendai West Park" => (38.26098, 140.86235),
"Transshipment-location Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-location Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-location Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-location Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-location Mikamine Park" => (38.2248, 140.8574),
"Transshipment-location Moomin Nanakita Park" => (38.31981, 140.87641))

shelters_dict = Dict("錦ケ丘小" => (38.25644, 140.76093),
"上杉山通小" => (38.27135, 140.87361),
"広瀬中" => (38.27389, 140.76242),
"仙台高" => (38.2821, 140.83406),
"仙台青陵中等教育" => (38.28862, 140.83386),
"榴岡小" => (38.26151, 140.89091),
"岩切小" => (38.29979, 140.94251),
"新田小" => (38.27628, 140.92843),
"宮城野中" => (38.26269, 140.90001),
"七郷小" => (38.24103, 140.94222),
"蒲町小" => (38.24245, 140.92912),
"東長町小" => (38.21892, 140.8892),
"富沢小" => (38.20972, 140.86297),
"長町中" => (38.23077, 140.87384),
"富沢中" => (38.21849, 140.86325),
"南光台小" => (38.30032, 140.9006),
"仙台商業高" => (38.32415, 140.87201),
"Transshipment-shelter Dainoharashinrinkoen" => (38.29598, 140.88088),
"Transshipment-shelter Sendai West Park" => (38.26098, 140.86235),
"Transshipment-shelter Miyaginohara Sports Park" => (38.25816, 140.90253),
"Transshipment-shelter Tsutsujigaoka Park" => (38.26067, 140.89702),
"Transshipment-shelter Hirosegawa Wakabayashi Ryokuchi Park" => (38.22769, 140.89955),
"Transshipment-shelter Hirosegawanakagawara Ryokuchi Park" => (38.22538, 140.90411),
"Transshipment-shelter Hirosegawahachihonmatsu Ryokuchi Park" => (38.22766, 140.89715),
"Transshipment-shelter Hirose River Iida Green Space" => (38.22359, 140.90396),
"Transshipment-shelter Mikamine Park" => (38.2248, 140.8574),
"Transshipment-shelter Moomin Nanakita Park" => (38.31981, 140.87641))

# Calculate and store distances in a nested dictionary
distances_dict = Dict{String, Dict{String, Float64}}()

for (sh_name, sh_coords) in shelters_dict
    distances_dict[sh_name] = Dict{String, Float64}()
    
    for (loc_name, loc_coords) in locations_dict
        distances_dict[sh_name][loc_name] = distance(sh_coords[1], sh_coords[2], loc_coords[1], loc_coords[2])
    end
end

# Print the calculated distances grouped by shelter
for (sh_name, distances) in distances_dict
    println("Distances from $sh_name:")
    
    for (loc_name, distance) in distances
        println("  $loc_name: $distance km")
    end
    
    println()
end


Distances from Transshipment-shelter Sendai West Park:
  Sendai Trust Tower: 1.3900432911278033 km
  Transshipment-location Dainoharashinrinkoen: 4.212489669577508 km
  Zuihōden Temple: 1.2000022924569043 km
  Osaki Hachiman Shrine: 2.0467339029135827 km
  Transshipment-location Miyaginohara Sports Park: 3.768294459183986 km
  Tohoku University Katahira Campus: 1.3519696730601776 km
  AER: 1.7495081791019689 km
  Tohoku Electric Power Company: 1.5714556919781841 km
  Transshipment-location Mikamine Park: 4.016679840373975 km
  Sendai Izumi PREMIUM OUTLETS: 9.205926648198131 km
  Sendai Mediatheque: 0.5737442088516905 km
  Sendai Station: 1.87306292669884 km
  Sendai Umino-Mori Aquarium: 11.229687997291052 km
  Transshipment-location Sendai West Park: 0.0 km
  Mitsui Outlet Park Sendai Port: 11.889918694773973 km
  Yurtec Stadium Sendai: 6.667949566695484 km
  Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park: 5.100322234234599 km
  Sendai Castle: 1.1508937863314395 km
  Trans

  Transshipment-location Miyaginohara Sports Park: 4.16271998009754 km
  Tohoku University Katahira Campus: 6.540921241934849 km
  AER: 6.171673749249518 km
  Tohoku Electric Power Company: 6.527938109733341 km
  Transshipment-location Mikamine Park: 8.284319925259895 km
  Sendai Izumi PREMIUM OUTLETS: 14.486666110351367 km
  Sendai Mediatheque: 7.609872739435999 km
  Sendai Station: 5.9735377623196015 km
  Sendai Umino-Mori Aquarium: 5.673103483193175 km
  Transshipment-location Sendai West Park: 7.769716396930641 km
  Mitsui Outlet Park Sendai Port: 5.792406597208983 km
  Yurtec Stadium Sendai: 10.19133278608103 km
  Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park: 4.320377143153531 km
  Sendai Castle: 8.187866819926143 km
  Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park: 4.53703073443607 km
  Transshipment-location Hirosegawanakagawara Ryokuchi Park: 4.021993276125499 km
  Rakuten Seimei Park Miyagi: 4.094937690723615 km
  Transshipment-location Moomin Nana

  Transshipment-location Sendai West Park: 3.5187436860264643 km
  Mitsui Outlet Park Sendai Port: 8.417867316332858 km
  Yurtec Stadium Sendai: 6.454348125018127 km
  Transshipment-location Hirosegawa Wakabayashi Ryokuchi Park: 3.85899850323957 km
  Sendai Castle: 4.298387059908913 km
  Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park: 3.8716441502641255 km
  Transshipment-location Hirosegawanakagawara Ryokuchi Park: 4.131955671356867 km
  Rakuten Seimei Park Miyagi: 0.6791506653777734 km
  Transshipment-location Moomin Nanakita Park: 6.643654221437625 km
  Transshipment-location Tsutsujigaoka Park: 0.3570027858096953 km
  Transshipment-location Hirose River Iida Green Space: 4.327224701670432 km

Distances from Transshipment-shelter Mikamine Park:
  Sendai Trust Tower: 3.920031004316817 km
  Transshipment-location Dainoharashinrinkoen: 8.134433508027914 km
  Zuihōden Temple: 2.9949466401755704 km
  Osaki Hachiman Shrine: 5.383349400684364 km
  Transshipment-location Miyag

  Transshipment-location Hirosegawahachihonmatsu Ryokuchi Park: 1.2253026531884001 km
  Transshipment-location Hirosegawanakagawara Ryokuchi Park: 1.5898492071932449 km
  Rakuten Seimei Park Miyagi: 4.378927245302598 km
  Transshipment-location Moomin Nanakita Park: 11.182112357673967 km
  Transshipment-location Tsutsujigaoka Park: 4.660425925233524 km
  Transshipment-location Hirose River Iida Green Space: 1.4997084363696678 km

Distances from 富沢小:
  Sendai Trust Tower: 5.310622147742059 km
  Transshipment-location Dainoharashinrinkoen: 9.64937781033672 km
  Zuihōden Temple: 4.539449852794495 km
  Osaki Hachiman Shrine: 7.119205976903372 km
  Transshipment-location Miyaginohara Sports Park: 6.495578504604057 km
  Tohoku University Katahira Campus: 4.973144667269801 km
  AER: 6.050465709316069 km
  Tohoku Electric Power Company: 6.319369215752202 km
  Transshipment-location Mikamine Park: 1.745361859027281 km
  Sendai Izumi PREMIUM OUTLETS: 14.761874174846113 km
  Sendai Mediatheque: 6